In [1]:
import pandas as pd
from function import *

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [2]:
nav_data = load_data(r"C:\Users\yueku\Desktop\VScode\Private_nav_research\data\SEC905_会世元丰CTA1号_2025-07-25.csv")

In [3]:
nav_df = nav_normalization(nav_data)
freq = infer_frequency(nav_df)
nav_df = date_normalization(nav_df, freq)

In [4]:
def intermediate_df(nav_df, benchmark_code):
    # 获取基准数据、合并数据
    start_day = nav_df["date"].min().strftime("%Y-%m-%d")
    end_day = nav_df["date"].max().strftime("%Y-%m-%d")
    benchmark_df = benchmark_data(benchmark_code, start_day, end_day)
    df = pd.merge(nav_df, benchmark_df, on="date", how="left")
    # 辅助数据
    ## df_nav
    df["excess_nav"] = df["nav_adjusted"] - df[benchmark_code] + 1
    df_nav = df[["date","nav_adjusted",benchmark_code,"excess_nav"]].round(4)
    df_nav.drop_duplicates(subset=["date"], inplace=True)
    ## df_return
    df["nav_return"] = df["nav_adjusted"].pct_change()
    df["benchmark_return"] = df[benchmark_code].pct_change()
    df[["nav_return", "benchmark_return"]] = df[["nav_return", "benchmark_return"]].fillna(0)
    df["excess_return"] = df["nav_return"] - df["benchmark_return"]
    df_return = df[["date", "nav_return", "benchmark_return", "excess_return"]].round(4)
    ## df_drawdown
    df_nav_copy = df_nav.copy()
    df_nav_copy["fun_max_so_far"] = df_nav_copy["nav_adjusted"].cummax()
    df_nav_copy["fund_drawdown"] = (df_nav_copy["nav_adjusted"] - df_nav_copy["fun_max_so_far"]) / df_nav_copy["fun_max_so_far"]
    df_nav_copy["benchmark_max_so_far"] = df_nav_copy[benchmark_code].cummax()
    df_nav_copy["benchmark_drawdown"] = (df_nav_copy[benchmark_code] - df_nav_copy["benchmark_max_so_far"]) / df_nav_copy["benchmark_max_so_far"]
    df_nav_copy["excess_max_so_far"] = df_nav_copy["excess_nav"].cummax()
    df_nav_copy["excess_drawdown"] = (df_nav_copy["excess_nav"] - df_nav_copy["excess_max_so_far"]) / df_nav_copy["excess_max_so_far"]
    df_drawdown = df_nav_copy[["date", "fund_drawdown", "benchmark_drawdown", "excess_drawdown"]].round(4)
    return df_nav, df_return, df_drawdown

In [21]:
df_nav, df_return, df_drawdown = intermediate_df(nav_df, "NH0100.NHF")

In [32]:
df_nav["nav_adjusted"].iloc[-1]

np.float64(2.1517)

In [ ]:
# 整体业绩指标
def calculate_overall_performance(df_nav, df_drawdown, df_return):
    days_diff = (df_nav["date"].max() - df_nav["date"].min()).days
    ## total_return
    total_return = df_nav["nav_adjusted"].iloc[-1] /df_nav["nav_adjusted"].iloc[0]- 1
    excess_total_return = df_nav["excess_nav"].iloc[-1] / df_nav["excess_nav"].iloc[0] - 1
    ## annual_return
    annual_return = pow(1 + total_return, 365 / days_diff) - 1
    excess_annual_return = pow(1 + excess_total_return, 365 / days_diff) - 1
    ## max_drawdown
    max_drawdown = df_drawdown["fund_drawdown"].min()
    excess_max_drawdown = df_drawdown["excess_drawdown"].min()
    ## annual_volatility
    annual_volatility = df_return["nav_return"].std() * np.sqrt(250 if freq == "D" else 52)
    excess_annual_volatility = df_return["excess_return"].std() * np.sqrt(250 if freq == "D" else 52)
    ## sharpe_ratio
    sharpe_ratio = (annual_return - 0.02) / annual_volatility
    excess_sharpe_ratio = (excess_annual_return - 0.02) / excess_annual_volatility
    list = [[total_return,annual_return,max_drawdown,annual_volatility,sharpe_ratio],[excess_total_return,excess_annual_return,excess_max_drawdown,excess_annual_volatility,excess_sharpe_ratio]]
    overall_performance = pd.DataFrame(list,columns=['总收益','年化收益','最大回撤','年化波动率','夏普比'],index=['基金','基准'])
    # 格式化
    for col in overall_performance.columns:
        overall_performance['夏普比'] = overall_performance['夏普比'].round(2)
        if col != '夏普比': 
            overall_performance[col] = overall_performance[col].map(lambda x: f"{x:.2%}")
    return overall_performance

In [38]:
calculate_overall_performance(df_nav, df_drawdown, df_return)

,总收益,年化收益,最大回撤,年化波动率,夏普比
基金,115.17%,11.60%,-17.56%,8.43%,1.14
基准,31.74%,4.03%,-43.19%,15.53%,0.13


In [7]:
overall_performance_dict = calculate_overall_performance(df_nav, df_drawdown, df_return)

NameError: name 'calculate_overall_performance' is not defined

In [6]:
# max_drawdown
def get_max_drawdown(df_nav, column_name):
    df_nav[f"{column_name}_max_so_far"] = df_nav[column_name].cummax()
    df_nav[f"{column_name}_drawdown"] = (
        df_nav[column_name] - df_nav[f"{column_name}_max_so_far"]
    ) / df_nav[f"{column_name}_max_so_far"]
    max_drawdown = df_nav[f"{column_name}_drawdown"].min()
    return max_drawdown

In [ ]:
def calculate_annual_performance(df_nav: pd.DataFrame, benchmark_code: str) -> pd.DataFrame:
    # 确保日期是datetime类型并按日期排序（避免inplace修改）
    df_nav = df_nav.sort_values('date').copy()  # 只在需要时复制一次
    # 获取每年最后一天的记录
    year_end_nav = df_nav.groupby(df_nav['date'].dt.year).last()
    # 初始化结果DataFrame
    results = []
    years = sorted(df_nav['date'].dt.year.unique())
    for i, year in enumerate(years):
        year_data = df_nav[df_nav['date'].dt.year == year]
        # 计算年初价值
        if i == 0:
            fund_start = year_data['nav_adjusted'].iloc[0]
            bench_start = year_data[benchmark_code].iloc[0]
        else:
            fund_start = year_end_nav.loc[years[i-1]]['nav_adjusted']
            bench_start = year_end_nav.loc[years[i-1]][benchmark_code]
        # 计算年末价值和收益率
        fund_end = year_data['nav_adjusted'].iloc[-1]
        bench_end = year_data[benchmark_code].iloc[-1]
        fund_return = fund_end / fund_start - 1
        bench_return = bench_end / bench_start - 1
        excess_return = fund_return - bench_return
        # 计算最大回撤
        fund_mdd = get_max_drawdown(year_data, 'nav_adjusted')
        bench_mdd = get_max_drawdown(year_data, benchmark_code)
        results.append({
            '分年度业绩': year,
            '基金收益': fund_return,
            '基金最大回撤': fund_mdd,
            '基准收益': bench_return,
            '基准最大回撤': bench_mdd,
            '超额收益': excess_return
        })
    # 创建结果DataFrame并格式化
    yearly_rtn = pd.DataFrame(results)
    yearly_rtn.set_index('分年度业绩', inplace=True)
    yearly_rtn.index.name = None
    # 更高效的小数处理方式（避免重复round）
    yearly_rtn = yearly_rtn.applymap(lambda x: f"{round(x, 4):.2%}")
    return yearly_rtn

In [18]:
calculate_annual_performance(df_nav, "NH0100.NHF")

C:\Users\yueku\AppData\Local\Temp\ipykernel_27108\665269327.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nav[f"{column_name}_max_so_far"] = df_nav[column_name].cummax()
C:\Users\yueku\AppData\Local\Temp\ipykernel_27108\665269327.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nav[f"{column_name}_drawdown"] = (
C:\Users\yueku\AppData\Local\Temp\ipykernel_27108\665269327.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,基金收益,基金最大回撤,基准收益,基准最大回撤,超额收益
2018,0.15%,-2.00%,-6.38%,-10.98%,6.53%
2019,13.80%,-5.29%,15.47%,-4.21%,-1.67%
2020,30.16%,-4.22%,7.51%,-16.07%,22.65%
2021,9.05%,-7.13%,20.94%,-10.76%,-11.89%
2022,6.16%,-11.50%,19.66%,-22.72%,-13.50%
2023,-1.83%,-9.94%,6.21%,-10.98%,-8.04%
2024,18.49%,-3.52%,-2.08%,-14.10%,20.57%
2025,7.71%,-2.66%,4.87%,-9.64%,2.85%


In [31]:
def calculate_monthly_performance(df_nav):
    """
    目前只支持月度胜率统计
    """
    # 找到日期序列中每个月的最后一天,并求月度收益
    monthly_rtn = df_nav.sort_values("date").groupby(df_nav["date"].dt.to_period("M")).tail(1).reset_index(drop=True)
    monthly_rtn["rtn"] = monthly_rtn["nav_adjusted"] / monthly_rtn["nav_adjusted"].shift(1)-1
    monthly_rtn.iloc[0, monthly_rtn.columns.get_loc("rtn")] = (monthly_rtn.iloc[0]["nav_adjusted"] - 1)
    # 生成月度收益表
    monthly_rtn["year"] = monthly_rtn["date"].dt.year
    monthly_rtn["month"] = monthly_rtn["date"].dt.month
    monthly_rtn = monthly_rtn.pivot_table(index="year", columns="month", values="rtn", aggfunc="sum")
    monthly_rtn.columns = [f"{x}月" for x in monthly_rtn.columns]
    monthly_rtn.index.name = None
    # 计算年度总收益和月度胜率
    monthly_rtn["年度总收益"] = monthly_rtn.apply(lambda x: np.prod(x + 1) - 1, axis=1)
    monthly_rtn["月度胜率"] = monthly_rtn.apply(lambda x: (x >= 0).sum() / (~np.isnan(x)).sum(), axis=1)
    # 保留四位小数
    monthly_rtn = monthly_rtn.map(lambda x: round(x, 4))
    for col in monthly_rtn.columns:
        monthly_rtn[col] = monthly_rtn[col].map(lambda x: f"{x:.2%}")
    # 将NAN变成NULL
    if (monthly_rtn.iloc[0, :] == "nan%").sum() + 3 == monthly_rtn.shape[1] and monthly_rtn.iloc[0, monthly_rtn.shape[1] - 3] == "0.000%":
        monthly_rtn = monthly_rtn.iloc[1:]
    return monthly_rtn.replace("nan%", "")

In [32]:
calculate_monthly_performance(df_nav)

,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,年度总收益,月度胜率
2018,,,,,,,,-0.24%,-0.58%,1.01%,-0.17%,0.14%,0.15%,50.00%
2019,0.07%,1.54%,1.75%,4.84%,3.25%,4.19%,-1.28%,-0.47%,-1.96%,0.62%,0.22%,0.46%,13.80%,76.92%
2020,1.81%,0.24%,6.52%,-1.58%,2.54%,3.36%,6.66%,-0.44%,0.78%,-1.05%,5.41%,2.84%,30.16%,76.92%
2021,-0.47%,4.14%,0.38%,0.21%,2.50%,-2.53%,0.17%,-0.21%,4.38%,0.03%,0.48%,-0.18%,9.05%,69.23%
2022,8.13%,1.84%,2.34%,0.50%,-0.61%,-4.78%,1.41%,1.08%,-3.42%,-1.65%,-2.74%,4.60%,6.16%,61.54%
2023,-1.11%,-3.53%,-3.31%,-3.14%,1.04%,-1.24%,4.11%,1.51%,0.50%,-1.03%,2.66%,2.04%,-1.83%,46.15%
2024,-1.73%,-2.05%,4.12%,2.47%,2.75%,-2.20%,2.61%,3.27%,2.70%,2.90%,0.56%,1.95%,18.49%,76.92%
2025,-2.06%,0.47%,0.02%,2.72%,3.32%,0.62%,2.50%,,,,,,7.71%,87.50%


In [ ]:
# nav ratio
def nav_ratio(nav_df, benchmark_code, freq):
    # 中间变量
    df_nav, df_return, df_drawdown = nav_ratio(nav_df, benchmark_code)
    # 整体业绩
    overall_performance_dict = calculate_overall_performance(df_nav, df_drawdown, df_return)
    # 分年度业绩
    annual_performance_df = calculate_annual_performance(df_nav, benchmark_code)
    # 分月度业绩
    data = df_nav[["date", "nav_adjusted"]]
    month_return = win_ratio_stastics(nav=data["nav_adjusted"],date=data["date"])
    month_return.index.name = "分月度业绩"
    month_return_table = month_return.reset_index(drop=False)

In [12]:
nav_ratio(nav_df, "NH0100.NHF", "w")

{'annual_return': 0.11601710437493251,
 'annual_volatility': np.float64(0.08429207693922128),
 'excess_annual_return': 0.04027822429987582,
 'excess_annual_volatility': np.float64(0.15527551787512914),
 'sharpe_ratio': np.float64(1.1390999944652633),
 'excess_sharpe_ratio': np.float64(0.1305951161997305)}

In [ ]:
# drawdown table
def get_drawdown_table(df_nav, df_drawdown, threshold):    
    # 初始化
    drawdowns = []
    start_idx = None
    end_idx = None
    peak_value = None
    for i in range(1, len(df_nav)):
        if start_idx is None and df_drawdown["fund_drawdown"].iloc[i] < 0:
            # 回撤开始
            start_idx = i - 1
            peak_value = df_nav["nav_adjusted"].iloc[i - 1]
        elif start_idx is not None:
            # 检查是否回升到回撤前的水平
            if df_nav["nav_adjusted"].iloc[i] >= peak_value:
                # 回补结束
                end_idx = i
                # 记录回撤信息
                drawdown_start_date = df_drawdown["date"].iloc[start_idx]
                drawdown_end_date = df_drawdown["date"].iloc[
                    np.argmin(df_drawdown["fund_drawdown"][start_idx:end_idx])
                    + start_idx
                ]
                reset_end_date = df_drawdown["date"].iloc[end_idx]
                fund_drawdown_percentage = df_drawdown["fund_drawdown"].iloc[
                    np.argmin(df_drawdown["fund_drawdown"][start_idx:end_idx])
                    + start_idx
                ]
                drawdowns.append(
                    (
                        drawdown_start_date,
                        drawdown_end_date,
                        reset_end_date,
                        fund_drawdown_percentage,
                    )
                )
                # 重置变量以寻找下一个回撤
                start_idx = None
                peak_value = None
    # 检查遍历结束后是否还有未完成的回撤
    if start_idx is not None:
        # 假设回撤结束于数据的最后一天
        end_idx = len(df_drawdown) - 1
        drawdown_start_date = df_drawdown["date"].iloc[start_idx]
        drawdown_end_date = df_drawdown["date"].iloc[
            np.argmin(df_drawdown["fund_drawdown"][start_idx:end_idx])
            + start_idx
        ]
        # 注意：这里我们假设没有回补，所以reset_end_date就是数据的最后一天
        reset_end_date = df_drawdown["date"].iloc[end_idx]
        fund_drawdown_percentage = df_drawdown["fund_drawdown"].iloc[
            np.argmin(df_drawdown["fund_drawdown"][start_idx:end_idx])
            + start_idx
        ]
        drawdowns.append(
            (
                drawdown_start_date,
                drawdown_end_date,
                reset_end_date,
                fund_drawdown_percentage,
            )
        )
    data_drawdown = pd.DataFrame(
        drawdowns, columns=["回撤开始时间", "回撤结束时间", "回补结束时间", "回撤"]
    )
    data_drawdown["回撤天数"] = (
        data_drawdown["回撤结束时间"] - data_drawdown["回撤开始时间"]
    )
    data_drawdown["回补天数"] = (
        data_drawdown["回补结束时间"] - data_drawdown["回撤结束时间"]
    )
    filtered_data = data_drawdown[data_drawdown["回撤"] < threshold]
    filtered_data.loc[
        filtered_data["回补结束时间"] == df_drawdown["date"].max(),
        ["回补结束时间", "回补天数"],
    ] = ""
    filtered_data["回撤"] = filtered_data["回撤"].map(lambda x: f"{x*100:.2f}%")
    drawdown_table = filtered_data
    return drawdown_table

In [42]:
get_drawdown_table(df_nav, df_drawdown)

C:\Users\yueku\AppData\Local\Temp\ipykernel_27108\1211282602.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["回撤"] = filtered_data["回撤"].map(lambda x: f"{x*100:.2f}%")


,回撤开始时间,回撤结束时间,回补结束时间,回撤,回撤天数,回补天数
19,2022-06-10,2023-06-09,2024-08-16,-17.56%,364 days,434 days
